In [1]:
version = 10

In [2]:
import tensorflow as tf
import os

# set tensorflow to be less verbose
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("Usandor Tensorflow version " + tf.__version__)

if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")


2023-11-25 21:52:20.703192: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-25 21:52:20.908271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 21:52:21.529807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Usandor Tensorflow version 2.13.1
Usando GPU: /device:GPU:0


2023-11-25 21:52:22.356721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 21:52:22.458837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 21:52:22.459107: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets
from keras.callbacks import EarlyStopping
from datetime import datetime
from empresa4.core import calculate_error_2

# Data Preparation

In [4]:
# Read the data
df = get_dataset("02_productos_todos_anti_leak")
df

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0
...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0


In [5]:
cluster_df = pd.read_csv("product_ids_clusters.csv")
cluster_df

{'axis': -1, 'kind': None, 'order': None}
()


,product_id,201701_norm,201702_norm,201703_norm,201704_norm,201705_norm,201706_norm,201707_norm,201708_norm,201709_norm,...,201806_norm,201807_norm,201808_norm,201809_norm,201810_norm,201811_norm,201812_norm,201901_norm,201902_norm,cluster
0,20001,0.091342,0.347663,0.567846,0.466153,0.654484,0.662267,0.449035,0.552176,0.573766,...,0.501371,0.640632,0.784656,0.626804,1.000000,0.789908,0.647724,0.555827,0.548559,0
1,20002,0.035110,0.286313,0.472443,0.295634,0.477368,0.547960,0.478475,0.350738,0.602969,...,0.585130,0.553193,0.657610,0.540080,0.780210,1.000000,0.571334,0.716985,0.590329,0
2,20003,0.375239,0.383889,0.468175,0.268330,0.316705,0.379824,0.400778,0.441167,0.748375,...,0.337259,0.334958,0.488019,0.465740,0.670507,0.616162,0.392968,0.492512,0.387094,0
3,20004,0.164645,0.400644,0.386223,0.403683,0.428377,0.465553,0.449286,0.822024,0.993244,...,0.353047,0.515169,0.738344,0.748157,0.638388,0.632612,0.461655,0.403118,0.348204,0
4,20005,0.179476,0.441811,0.451804,0.530902,0.413083,0.423504,0.501455,0.855840,1.000000,...,0.438761,0.402467,0.701053,0.610395,0.716162,0.375956,0.298511,0.291258,0.328378,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,21295,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
1292,21296,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1293,21297,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
1294,21298,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [6]:
# add column cluster from cluster_df to df
df = df.merge(cluster_df[["product_id", "cluster"]], on='product_id', how='left')
df["cluster"].isna().sum()

0

In [7]:
equiv = {
    201701: 201703,
    201702: 201704,
    201703: 201705,
    201704: 201706,
    201705: 201707,
    201706: 201708,
    201707: 201709,
    201708: 201710,
    201709: 201711,
    201710: 201712,
    201711: 201801,
    201712: 201802,
    201801: 201803,
    201802: 201804,
    201803: 201805,
    201804: 201806,
    201805: 201807,
    201806: 201808,
    201807: 201809,
    201808: 201810,
    201809: 201811,
    201810: 201812,
    201811: 201901,
    201812: 201902,
    201901: 201903,
    201902: 201904,
    201903: 201905,
    201904: 201906,
    201905: 201907,
    201906: 201908,
    201907: 201909,
    201908: 201910,
    201909: 201911,
    201910: 201912,
    201911: 202001,
    201912: 202002,
    202001: 202003,
    202002: 202004,
}
target_df = get_dataset("02_productos_todos")


# Filter data up to 201902
def lag_target_class(row):
    # from the column "periodo" and "product_id" of this row, get the equivalen periodo in equiv and get the tn column from "target" df for this product_id and the equiv periodo
    product_id = row["product_id"]
    periodo = row["periodo"]
    periodo_equiv = equiv.get(periodo)
    if periodo_equiv is None:
        return None
    value = target_df[(target_df["product_id"] == product_id) & (target_df["periodo"] == periodo_equiv)]["tn"].values[0]
    return value

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


In [8]:
df["target"] = df.apply(lag_target_class, axis=1)
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,cluster,target
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,0,1303.35771
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0,0,834.73521
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0,0,917.16548
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0,0,489.91328
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0,0,563.89955
...,...,...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0,1,0.00000
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0,1,0.00000
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0,1,0.00000
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0,1,0.00000


In [9]:
# Sort by `periodo` just to be sure
product_data = df.sort_values(["product_id", "periodo"])

In [10]:
# Using scikit-learn, ONE HOT ENCODE the categorical variables: product_category
product_data = pd.get_dummies(product_data, columns=["product_category", "cat2"])

# sort columns so that 'lag_tn' is the last column in the dataframe
product_data = product_data[[col for col in product_data.columns if col != 'target'] + ['target']]

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


In [11]:
columns = product_data.columns
product_category_columns = [col for col in columns if col.startswith("product_category")]

# convert boolean columns to int
product_data = product_data.astype({col: 'int32' for col in product_category_columns})

In [12]:
# convert bool columns to int
cat2_columns = [col for col in columns if col.startswith("cat2")]

product_data = product_data.astype({col: 'int32' for col in cat2_columns})
product_data.dtypes.sort_values()

{}
()


cat2_CABELLO                  int32
cat2_OTROS                    int32
cat2_HOGAR                    int32
cat2_DEOS                     int32
cat2_DENTAL                   int32
cat2_unknown                  int32
cat2_ADEREZOS                 int32
product_category_unknown      int32
product_category_REF          int32
cat2_PIEL1                    int32
product_category_PC           int32
product_category_FOODS        int32
cat2_PROFESIONAL              int32
cat2_ROPA ACONDICIONADOR      int32
cat2_ROPA LAVADO              int32
cat2_ROPA MANCHAS             int32
cat2_SOPAS Y CALDOS           int32
cat2_TE                       int32
cat2_VAJILLA                  int32
product_category_HC           int32
cat2_PIEL2                    int32
periodo                       int64
plan_precios_cuidados         int64
product_id                    int64
cluster                       int64
sku_size                    float64
tn                          float64
cust_request_tn             

In [13]:
product_data.isna().sum()

periodo                     0
product_id                  0
cust_request_qty            0
cust_request_tn             0
tn                          0
sku_size                    0
plan_precios_cuidados       0
cluster                     0
product_category_FOODS      0
product_category_HC         0
product_category_PC         0
product_category_REF        0
product_category_unknown    0
cat2_ADEREZOS               0
cat2_CABELLO                0
cat2_DENTAL                 0
cat2_DEOS                   0
cat2_HOGAR                  0
cat2_OTROS                  0
cat2_PIEL1                  0
cat2_PIEL2                  0
cat2_PROFESIONAL            0
cat2_ROPA ACONDICIONADOR    0
cat2_ROPA LAVADO            0
cat2_ROPA MANCHAS           0
cat2_SOPAS Y CALDOS         0
cat2_TE                     0
cat2_VAJILLA                0
cat2_unknown                0
target                      0
dtype: int64

In [14]:
product_data.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'sku_size', 'plan_precios_cuidados', 'cluster',
       'product_category_FOODS', 'product_category_HC', 'product_category_PC',
       'product_category_REF', 'product_category_unknown', 'cat2_ADEREZOS',
       'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS',
       'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL',
       'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS',
       'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_unknown',
       'target'],
      dtype='object')

In [15]:
product_data["periodo"].value_counts()

{}
()


periodo
201701    1296
201702    1296
201901    1296
201812    1296
201811    1296
201810    1296
201809    1296
201808    1296
201807    1296
201806    1296
201805    1296
201804    1296
201803    1296
201802    1296
201801    1296
201712    1296
201711    1296
201710    1296
201709    1296
201708    1296
201707    1296
201706    1296
201705    1296
201704    1296
201703    1296
201902    1296
Name: count, dtype: int64

# Train Test Validation Split

In [16]:
train_df = product_data[product_data["periodo"] <= 201811].copy()
validation_df = product_data[product_data["periodo"] == 201812].copy()
test_df = product_data[product_data["periodo"] == 201902].copy()

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


In [17]:
train_df.shape, validation_df.shape, test_df.shape

((29808, 30), (1296, 30), (1296, 30))

In [18]:
# standarize the data
from sklearn.preprocessing import StandardScaler

scaler_train = StandardScaler()
scaler_validation = StandardScaler()
scaler_test_product_id = StandardScaler()
scaler_test_target = StandardScaler()
scaler_test_rest = StandardScaler()

train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
validation_df[validation_df.columns] = scaler_validation.fit_transform(validation_df[validation_df.columns])

rest = [col for col in test_df.columns if col not in ["product_id", "target"]]

test_df[rest] = scaler_test_rest.fit_transform(test_df[rest])
test_df["target"] = scaler_test_target.fit_transform(test_df[["target"]])
test_df["product_id"] = scaler_test_product_id.fit_transform(test_df[["product_id"]])

{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'decimals': 0, 'out': None}
()
{'axis': -1, 'kind': 'mergesort', 'order': None}
()
{'axis': -1, 'mode': 'raise', 'out': None}
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),)


# Iterations Functions

In [19]:
def get_n_neurons(n_layer, n_neurons, embudo_1, embudo_2, embudo_3):
    if n_layer == 1:
        return n_neurons
    elif n_layer == 2:
        return int(n_neurons // embudo_1)
    elif n_layer == 3:
        return int(n_neurons // embudo_2)
    elif n_layer == 4:
        return int(n_neurons // embudo_3)
    else:
        return int(n_neurons // embudo_3)

def train_model(n_layers, n_neurons, dropout_threshold, embudo_1, embudo_2, embudo_3, X_train, y_train, X_validate, y_validate):
    print(f"Training model with {n_layers} layers, {n_neurons} neurons, {embudo_1} embudo_1, {embudo_2} embudo_2, and {embudo_3} embudo_3.")

    # Reshape the input data to have three dimensions: [samples, time steps, features]
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_validate = X_validate.values.reshape((X_validate.shape[0], 1, X_validate.shape[1]))

    # Number of features (excluding the target variable)
    n_features = X_train.shape[2]
    print(f"Número de features: {n_features}")

    # Define the LSTM model structure based on the number of layers
    model = Sequential()
    for i in range(n_layers):
        if i == n_layers - 1: # Last layer
            model.add(LSTM(n_neurons, activation="relu", input_shape=(1, n_features)))
        else: # Intermediate layers
            final_n_neurons = get_n_neurons(i, n_neurons, embudo_1, embudo_2, embudo_3)
            model.add(LSTM(final_n_neurons, activation="relu", return_sequences=True, input_shape=(1, n_features)))
            model.add(Dropout(dropout_threshold))
    
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model
    model.fit(X_train, y_train, epochs=200, validation_data=(X_validate, y_validate), callbacks=[early_stopping], verbose=1)
    return model

In [20]:
def make_predictions(model, X_test, trial_number, cluster = None):
    rows = []
    # for each product_id, generate a prediction for 201904
    for i, product in enumerate(X_test["product_id"].unique()):
        print(
            f"Predicting 201904 for product {product} ({i+1}/{len(X_test['product_id'].unique())}))"
        )

        this_X_test = X_test[X_test["product_id"] == product]
        this_X_test = this_X_test.values.reshape((this_X_test.shape[0], 1, this_X_test.shape[1]))


        # Make prediction
        yhat = model.predict(this_X_test, verbose=0)
        if yhat[0][0] < 1:
            yhat[0][0] = 0

        # Append to final output DataFrame
        rows.append(
            {
                "product_id": product,
                "target": yhat[0][0],
            }
        )
        
    final_output = pd.DataFrame(
        rows,
        columns=["product_id", "target"],
    )
    
    final_output["product_id"] = scaler_test_product_id.inverse_transform(final_output[["product_id"]])
    final_output["target"] = scaler_test_target.inverse_transform(final_output[["target"]])
    final_output["product_id"] = final_output["product_id"].astype(int)

    final_output = final_output.sort_values("product_id", ascending=True)
    timestamp = datetime.now().timestamp()
    final_output.to_csv(f"./output/output_lstm_{version}_BO_{cluster if cluster else 'TODO'}_{trial_number}_{timestamp}.csv", index=False)
    return final_output

In [21]:
def calculate_score(predictions):
    # revert the standarization for the output

    
    predictions.rename(columns={"target": "prediction"}, inplace=True)
    
    return calculate_error_2(predictions, 201904)


# Bayesian Optimization

In [22]:
import pandas as pd
import optuna
from optuna import Trial

/home/gero/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
def objective(trial: Trial, cluster:int = None):
    # get trial number
    trial_number = trial.number

    # Define the search space using Optuna
    n_layers = trial.suggest_int('n_layers', 1, 4)
    n_neurons = trial.suggest_int('n_neurons', 20, 500)
    embudo_1 = trial.suggest_float('embudo_1', 0.9, 1.5)
    embudo_2 = trial.suggest_float('embudo_2', 0.8, 2.5)
    embudo_3 = trial.suggest_float('embudo_3', 0.7, 3.5)
    dropout_threshold = trial.suggest_float('dropout_threshold', 0.1, 0.66)

    if cluster is not None:
        train_df_cluster = train_df[train_df["cluster"] == cluster].copy()
        validation_df_cluster = validation_df[validation_df["cluster"] == cluster].copy()
        test_df_cluster = test_df[test_df["cluster"] == cluster].copy()

        X_train, y_train = train_df_cluster.drop(["target"], axis=1), train_df_cluster["target"]
        X_validate, y_validate = validation_df_cluster.drop(["target"], axis=1), validation_df_cluster["target"]
        X_test = test_df_cluster.drop(["target"], axis=1)
    else:
        X_train, y_train = train_df.drop(["target"], axis=1), train_df["target"]
        X_validate, y_validate = validation_df.drop(["target"], axis=1), validation_df["target"]
        X_test = test_df.drop(["target"], axis=1)

    model = train_model(n_layers, n_neurons, dropout_threshold, embudo_1, embudo_2, embudo_3, X_train, y_train, X_validate, y_validate)
    predictions = make_predictions(model, X_test, trial_number, cluster)
    score = calculate_score(predictions)

    # Save the results
    result = {
        "n_layers": n_layers,
        "n_neurons": n_neurons,
        "embudo_1": embudo_1,
        "embudo_2": embudo_2,
        "embudo_3": embudo_3,
        "dropout_threshold": dropout_threshold,
        "score": score
    }
    g_s_df = pd.DataFrame([result])
    g_s_df.to_csv(f"./output/lstm_{version}_optuna_{n_layers}_{n_neurons}_{dropout_threshold}_{score}.csv", mode='a', index=False)

    return score

In [24]:
def bo(cluster=None):

    # Create a study object and specify the optimization direction
    study = optuna.create_study(direction='minimize', study_name=f"lstm_{version}_BO_{cluster if cluster else 'TODO'}")

    # Perform optimization
    study.optimize(lambda x: objective(x, cluster), n_trials=50)

    # Extract the best trial
    best_trial = study.best_trial

    print(f"Best trial{' for cluster' + str(cluster) if cluster else ''}:")
    for key, value in best_trial.params.items():
        print(f"{key}: {value}")

In [30]:
train_df["cluster"].unique()

array([-0.8078684 , -0.02005994,  0.76774853,  2.34336546,  1.55555699])

In [29]:
for cluster in train_df["cluster"].unique()[1:]:
    print(f"Training for cluster {cluster}")
    bo(cluster)
    print(f"Finished training for cluster {cluster}")

[I 2023-11-25 21:56:00,002] A new study created in memory with name: lstm_10_BO_-0.020059937789255886


Training for cluster -0.020059937789255886
{'out': None}
(1, 4)
{'out': None}
(20, 500)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
Training model with 3 layers, 434 neurons, 1.4262970651291207 embudo_1, 2.4372122423922833 embudo_2, and 1.226984633715104 embudo_3.
Número de features: 29


2023-11-25 21:56:00.016076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 21:56:00.016209: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 21:56:00.016268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/200


2023-11-25 21:56:02.712627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-25 21:56:02.725656: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f35a40017a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-25 21:56:02.725666: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2023-11-25 21:56:02.734253: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-25 21:56:02.860896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2023-11-25 21:56:02.951734: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

...

Predicting 201904 for product 1.719817299453615 (157/158))
Predicting 201904 for product 1.7251486248922152 (158/158))
{}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08888888888888889, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(68.71428571428571, 481.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(10.688888888888888, 481.0)
{'decimals': 0, 'out': None}
()


[I 2023-11-25 23:01:52,487] Trial 48 finished with value: 8.997037993765232 and parameters: {'n_layers': 1, 'n_neurons': 169, 'embudo_1': 1.0899674133374442, 'embudo_2': 1.7074846925725717, 'embudo_3': 2.4020483633958634, 'dropout_threshold': 0.4973273819915169}. Best is trial 0 with value: 8.997037993765232.


{}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08695652173913043, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(68.71428571428571, 481.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(10.456521739130435, 481.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 500)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 500)
{'axis': None, 'out': None}
()
{'axis':

[I 2023-11-25 23:02:05,783] Trial 49 finished with value: 8.997037993765232 and parameters: {'n_layers': 2, 'n_neurons': 210, 'embudo_1': 1.0080702960222416, 'embudo_2': 2.494871233456238, 'embudo_3': 3.285956960528645, 'dropout_threshold': 0.352090714195068}. Best is trial 0 with value: 8.997037993765232.


Predicting 201904 for product 1.7251486248922152 (158/158))
{}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
{'axis': None, 'dtype': None, 'out': None}
()
Best trial for cluster1.5555569940213847:
n_layers: 4
n_neurons: 308
embudo_1: 1.3013974395521934
embudo_2: 2.210097618703375
embudo_3: 3.0174157779070008
dropout_threshold: 0.20751666592540416
Finished training for cluster 1.5555569940213847
